In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm as tn
from bs4 import BeautifulSoup as bs
from urllib.request import urlretrieve

import pandas as pd
import time

## 네이버 이미지 수집

In [10]:
url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=%EC%B9%B4%EC%B9%B4%EC%98%A4+%EB%9D%BC%EC%9D%B4%EC%96%B8'

driver = webdriver.Chrome()

driver.get(url)

In [11]:
for i in range(15) :
    driver.find_element_by_css_selector('body').send_keys(Keys.SPACE)
    time.sleep(2)

In [12]:
html = bs(driver.page_source, 'html.parser')
driver.quit()

In [13]:
# 이미지 요소를 수집
imgs = html.select('img._image._listImage')

len(imgs)

240

## images 폴더에 네이버에서 검색한 이미지 저장
- 경로: images/파일명.확장자

In [14]:
src_list = []
cnt = 0
for img in imgs :
    # 요소의 속성값 접근
    if img.get('src')[0] == 'h' :
        # print(img.get('src'))
        src_list.append(img.get('src'))
    else :
    # if 'data:' in img.get('src');
        continue

In [15]:
print(len(src_list))

181


In [16]:
for i in range(len(src_list)) :
    start = src_list[i].rfind('.')
    end = src_list[i].rfind('&')
    urlretrieve(src_list[i], 'images/이미지{}.{}'.format(i,src_list[i][start+1:end]))

## 유튜브 데이터 수집
- 영상제목, 영상주소, 조회수, 썸네일 주소 수집
- 유튜브_영상정보.csv 파일로 저장
- 영상주소의 경우 'https://www.youtube.com' 메인주소를 붙인후 저장할 것
- aria-label속성에 있는 조회수를 추출

In [17]:
url = 'https://www.youtube.com/results?search_query=%EA%B3%A0%EC%96%91%EC%9D%B4'

driver = webdriver.Chrome()

driver.get(url)

In [18]:
for i in range(15) :
    driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [19]:
html = bs(driver.page_source, 'html.parser')
driver.quit()

In [20]:
vid_titles = html.select('a#video-title')

In [21]:
len(vid_titles)

57

In [22]:
vid_titles[0].text.strip() # 영상제목

'아직도 젖먹는 아기 고양이가 여기있어요!'

In [23]:
vid_titles[0].get('href') # 영상주소

'/watch?v=zdpORWFBObs'

In [24]:
start = vid_titles[0].get('aria-label').rfind('회수')
end = vid_titles[0].get('aria-label').rfind('회')

In [25]:
vid_titles[0].get('aria-label')[start+3:end] # 조회수

'271,854'

In [26]:
thumbs = html.select('a#thumbnail img')

In [27]:
thumbs[55].get('src')

'https://i.ytimg.com/vi/wQ7WRFijgeA/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLAjRKO_pLHM5G29grE-rOz30w6LLQ'

In [28]:
thumbs

[<img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/zdpORWFBObs/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLCghzSvKLeaapFjHGxVGeSsz8IaRw" width="360"/>,
 <img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/1Tv3idOf8tE/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLAzfRoppbPXuM6laaP7eWWbyiz1ZQ" width="360"/>,
 <img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/Qi5c4UGrl_E/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLAm8lujklYE4-d4s93Wm2LY8pIRqA" width="360"/>,
 <img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/bQpHtHabTPQ/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLDpxDqXzi3dQXgpCBWiRgWX8fng9A" width="360"/>,
 <img alt="" class="style-scope yt-img-shadow" id="img" src="https://i.ytimg.com/vi/JveHwI22PIY/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSF

In [29]:
len(thumbs)

58

In [30]:
title_l = []
mAddr_l = []
view_l = []
thumb_l = []

for i in range(0,55) :
    title_l.append(vid_titles[i].text.strip())
    mAddr_l.append('https://www.youtube.com'+vid_titles[i].get('href'))
    
    start = vid_titles[i].get('aria-label').rfind('회수')
    end = vid_titles[i].get('aria-label').rfind('회')
    view_l.append(vid_titles[i].get('aria-label')[start+3:end])
    
    thumb_l.append(thumbs[i].get('src'))

In [31]:
from pandas import DataFrame

In [32]:
df = DataFrame({"영상제목":title_l, "영상주소":mAddr_l, "조회수":view_l, 
                "영상주소":thumb_l})
df

,영상제목,영상주소,조회수
0,아직도 젖먹는 아기 고양이가 여기있어요!,https://i.ytimg.com/vi/zdpORWFBObs/hq720.jpg?s...,"271,854"
1,고양이가 괴물을 만났습니다.,https://i.ytimg.com/vi/1Tv3idOf8tE/hq720.jpg?s...,"58,967"
2,그들의 첫 항문낭 짜기...... 충격적인 시간이었다,https://i.ytimg.com/vi/Qi5c4UGrl_E/hq720.jpg?s...,"70,319"
3,간식 있는 척 장난 쳤더니 종 치면서 짜증내는 고양이,https://i.ytimg.com/vi/bQpHtHabTPQ/hq720.jpg?s...,"24,200"
4,구조한 길 고양이 금순이 입양 갔습니다. 최종 ep ㅣ 진돗개,https://i.ytimg.com/vi/JveHwI22PIY/hqdefault.j...,"6,986"
5,[삐약모음]자꾸 삐약! 삐약! 집사한테 말 거는 고양이 미루ㅎ,https://i.ytimg.com/vi/yrGbpikFHDI/hq720.jpg?s...,"17,665"
6,식음 전폐의 상황에서 성공한 고양이 열마리 합사 2년간의 기록,https://i.ytimg.com/vi/DMZ1KWiHCCk/hq720.jpg?s...,"19,560"
7,아! 진짜 강아지가 고양이로 환생한 것 같아요ㅋ,https://i.ytimg.com/vi/4BwNZSTHjCQ/hq720.jpg?s...,"908,356"
8,장난감뱀에 진심인 고양이들 반응ㅋㅋㅋ,https://i.ytimg.com/vi/bxEq4zIyy0E/hq720.jpg?s...,"6,124,025"
9,새해 맞이 강아지 고양이 형제 동시 목욕의 날 │이웃집의 백호│웰시코기 백호,https://i.ytimg.com/vi/vnz1Z4OgpLA/hq720.jpg?s...,"25,231"


In [33]:
df.to_csv('유튜브_영상정보.csv', encoding = 'utf-8-sig')